In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
key_path = './service_account/gentle-keyword-423715-j0-03be08ad6412.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

In [4]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project=credentials.project_id
)

**IMPORT DATA**

In [54]:
df_prod = pd.read_csv('./top-product.csv')

for i in ['total_hits', 'total_visits', 'total_page_views', 'total_first_visits', 'total_time_on_site', 'total_transactions']:
    df_prod[i] = df_prod[i].astype('int')

print(df_prod.info())
df_prod.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   category            82 non-null     object 
 1   product             82 non-null     object 
 2   total_hits          82 non-null     int32  
 3   total_visits        82 non-null     int32  
 4   total_page_views    82 non-null     int32  
 5   total_first_visits  82 non-null     int32  
 6   total_time_on_site  82 non-null     int32  
 7   avg_time_on_site    82 non-null     float64
 8   total_transactions  82 non-null     int32  
 9   total_revenue       82 non-null     float64
 10  avg_revenue         82 non-null     float64
dtypes: float64(3), int32(6), object(2)
memory usage: 5.3+ KB
None


,category,product,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_site,avg_time_on_site,total_transactions,total_revenue,avg_revenue
0,Accessories,8 pc Android Sticker Sheet,7328,183,4905,84,186822,1138.402727,54,1.237723e+10,2.224186e+08
1,Accessories,Android Sticker Sheet Ultra Removable,6907,232,4782,140,210288,993.763405,45,5.401255e+09,9.633929e+07
2,Accessories,Basecamp Explorer Powerbank Flashlight,2791,81,2039,49,97267,1200.827160,13,3.518450e+09,2.706500e+08
3,Accessories,Bottle Opener Clip,5474,134,3996,66,121723,908.380597,44,3.792220e+09,9.029095e+07
4,Accessories,Crunch Noise Dog Toy,6771,166,4644,70,157377,948.054217,56,2.667470e+09,5.129750e+07


In [55]:
# Get the schema of the result
schema = [
    bigquery.SchemaField("category", "STRING"),
    bigquery.SchemaField("product", "STRING"),
    bigquery.SchemaField("total_hits", "INTEGER"),
    bigquery.SchemaField("total_visits", "INTEGER"),
    bigquery.SchemaField("total_page_views", "INTEGER"),
    bigquery.SchemaField("total_first_visits", "INTEGER"),
    bigquery.SchemaField("total_time_on_site", "INTEGER"),
    bigquery.SchemaField("avg_time_on_site", "FLOAT"),
    bigquery.SchemaField("total_transactions", "INTEGER"),
    bigquery.SchemaField("total_revenue", "FLOAT"),
    bigquery.SchemaField("avg_revenue", "FLOAT"),
]

# Define the new table reference
project_id = credentials.project_id
dataset_id = 'dummy'
table_id = 'top-products'
table_ref = client.dataset(dataset_id, project=project_id).table(table_id)

# Delete the existing destination table if it exists
try:
    client.delete_table(table_ref)
    print(f"Deleted table {table_ref}")
except Exception as e:
    print(f"Table {table_ref} does not exist: {e}")
        
# Create a new table with the schema
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # API request


# Load DataFrame to BigQuery table
job = client.load_table_from_dataframe(df_prod, table_ref)

# Wait for the load job to complete
job.result()

print(f"Loaded {job.output_rows} rows into {dataset_id}:{table_id}.")

Deleted table gentle-keyword-423715-j0.dummy.top-products
Loaded 82 rows into dummy:top-products.


In [51]:
df_cat = pd.read_csv('./data_modeling.csv')

for i in ['total_hits', 'total_visits', 'total_page_views', 'total_first_visits', 'total_time_on_site', 'total_transactions']:
    df_cat[i] = df_cat[i].astype('int')

df_cat['date'] = pd.to_datetime(df_cat['date'])
print(df_cat.info())
df_cat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2170 entries, 0 to 2169
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   category                               2170 non-null   object        
 1   date                                   2170 non-null   datetime64[ns]
 2   total_hits                             2170 non-null   int32         
 3   total_visits                           2170 non-null   int32         
 4   total_page_views                       2170 non-null   int32         
 5   total_first_visits                     2170 non-null   int32         
 6   total_time_on_site                     2170 non-null   int32         
 7   avg_time_on_site                       2170 non-null   float64       
 8   total_transactions                     2170 non-null   int32         
 9   total_revenue                          2170 non-null   float64 

,category,date,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_site,avg_time_on_site,total_transactions,total_revenue,...,medium_not_set,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views
0,Accessories,2016-08-01,4566,89,3027,33,87104,563.195850,20,1.459300e+09,...,0,0,3,3,57,14,0,146,0,121
1,Accessories,2016-08-02,5619,94,3713,40,107344,815.891196,10,6.299200e+08,...,0,0,14,0,102,2,0,151,0,133
2,Accessories,2016-08-03,6343,113,4077,58,116221,986.585195,0,0.000000e+00,...,0,0,15,2,140,6,0,179,0,162
3,Accessories,2016-08-04,5006,95,3549,68,141102,805.315385,1,1.222667e+08,...,0,8,2,0,93,13,0,152,0,135
4,Accessories,2016-08-05,6448,110,4578,70,139769,799.491296,28,3.990420e+09,...,0,7,20,0,192,0,0,180,0,150


In [52]:
df_cat['category'].unique()

array(['Accessories', 'Electronics', "Men's", 'Office', 'Shop by Brand',
       "Women's"], dtype=object)

In [53]:
# Get the schema of the result
schema = [
    bigquery.SchemaField("category", "STRING"),
    bigquery.SchemaField("date", "DATE"),
    bigquery.SchemaField("total_hits", "INTEGER"),
    bigquery.SchemaField("total_visits", "INTEGER"),
    bigquery.SchemaField("total_page_views", "INTEGER"),
    bigquery.SchemaField("total_first_visits", "INTEGER"),
    bigquery.SchemaField("total_time_on_site", "INTEGER"),
    bigquery.SchemaField("avg_time_on_site", "FLOAT"),
    bigquery.SchemaField("total_transactions", "INTEGER"),
    bigquery.SchemaField("total_revenue", "FLOAT"),
    bigquery.SchemaField("avg_revenue", "FLOAT"),
    bigquery.SchemaField("medium_none", "INTEGER"),
    bigquery.SchemaField("medium_not_set", "INTEGER"),
    bigquery.SchemaField("medium_affiliate", "INTEGER"),
    bigquery.SchemaField("medium_cpc", "INTEGER"),
    bigquery.SchemaField("medium_cpm", "INTEGER"),
    bigquery.SchemaField("medium_organic", "INTEGER"),
    bigquery.SchemaField("medium_referral", "INTEGER"),
    bigquery.SchemaField("action_Check_out", "INTEGER"),
    bigquery.SchemaField("action_Click_through_of_product_lists", "INTEGER"),
    bigquery.SchemaField("action_Completed_purchase", "INTEGER"),
    bigquery.SchemaField("action_Product_detail_views", "INTEGER")
]

# Define the new table reference
project_id = credentials.project_id
dataset_id = 'dummy'
table_id = 'data-main'
table_ref = client.dataset(dataset_id, project=project_id).table(table_id)

# Delete the existing destination table if it exists
try:
    client.delete_table(table_ref)
    print(f"Deleted table {table_ref}")
except Exception as e:
    print(f"Table {table_ref} does not exist: {e}")
        
# Create a new table with the schema
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # API request


# Load DataFrame to BigQuery table
job = client.load_table_from_dataframe(df_cat, table_ref)

# Wait for the load job to complete
job.result()

print(f"Loaded {job.output_rows} rows into {dataset_id}:{table_id}.")

Deleted table gentle-keyword-423715-j0.dummy.data-main
Loaded 2170 rows into dummy:data-main.


In [43]:
# categories = ['Accessories', 'Electronics', 'Office', 'Men\'s', 'Women\'s', 'Shop by Brand']
# for cat in categories:
#     df_exp = pd.read_csv(f'./prediction-revenue-{cat}.csv')
#     df_exp['date'] = pd.to_datetime(df_exp['date'])
#     # Get the schema of the result
#     schema = [
#         bigquery.SchemaField("date", "DATE"),
#         bigquery.SchemaField("total_revenue", "FLOAT"),
#         bigquery.SchemaField("pred_total_revenue", "FLOAT"),
#         bigquery.SchemaField("category", "STRING"),
#     ]

#     # Define the new table reference
#     project_id = credentials.project_id
#     dataset_id = 'dummy'
#     name_table = cat.replace("'", "").replace(" ", "")
#     table_id = f'{name_table}'
#     table_ref = client.dataset(dataset_id, project=project_id).table(table_id)

#     # Delete the existing destination table if it exists
#     try:
#         client.delete_table(table_ref)
#         print(f"Deleted table {table_ref}")
#     except Exception as e:
#         print(f"Table {table_ref} does not exist: {e}")

#     # Create a new table with the schema
#     table = bigquery.Table(table_ref, schema=schema)
#     table = client.create_table(table)  # API request


#     # Load DataFrame to BigQuery table
#     job = client.load_table_from_dataframe(df_exp, table_ref)

#     # Wait for the load job to complete
#     job.result()

#     print(f"Loaded {job.output_rows} rows into {dataset_id}:{table_id}.")

Deleted table gentle-keyword-423715-j0.dummy.Accessories
Loaded 251 rows into dummy:Accessories.
Deleted table gentle-keyword-423715-j0.dummy.Electronics
Loaded 276 rows into dummy:Electronics.
Deleted table gentle-keyword-423715-j0.dummy.Office
Loaded 276 rows into dummy:Office.
Table gentle-keyword-423715-j0.dummy.Mens does not exist: 404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/gentle-keyword-423715-j0/datasets/dummy/tables/Mens?prettyPrint=false: Not found: Table gentle-keyword-423715-j0:dummy.Mens
Loaded 274 rows into dummy:Mens.
Table gentle-keyword-423715-j0.dummy.Womens does not exist: 404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/gentle-keyword-423715-j0/datasets/dummy/tables/Womens?prettyPrint=false: Not found: Table gentle-keyword-423715-j0:dummy.Womens
Loaded 264 rows into dummy:Womens.
Table gentle-keyword-423715-j0.dummy.ShopbyBrand does not exist: 404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/gentle-keyword-423715-j0

In [47]:
df = pd.DataFrame()
categories = ['Accessories', 'Electronics', 'Office', 'Men\'s', 'Women\'s', 'Shop by Brand']
for cat in categories:
    df_exp = pd.read_csv(f'./prediction-revenue-{cat}.csv')
    df = pd.concat([df, df_exp])

df['date'] = pd.to_datetime(df['date'])
print(df.info())
df.head()

,date,total_revenue,pred_total_revenue,category
0,2016-08-01,4.377900e+09,2.533989e+09,Accessories
1,2016-08-02,1.889760e+09,2.757827e+09,Accessories
2,2016-08-03,0.000000e+00,3.003203e+09,Accessories
3,2016-08-04,3.668000e+08,2.802602e+09,Accessories
4,2016-08-05,1.197126e+10,2.828253e+09,Accessories


In [49]:
# Get the schema of the result
schema = [
    bigquery.SchemaField("date", "DATE"),
    bigquery.SchemaField("total_revenue", "FLOAT"),
    bigquery.SchemaField("pred_total_revenue", "FLOAT"),
    bigquery.SchemaField("category", "STRING"),
]

# Define the new table reference
project_id = credentials.project_id
dataset_id = 'dummy'
name_table = cat.replace("'", "").replace(" ", "")
table_id = f'prediction-revenue'
table_ref = client.dataset(dataset_id, project=project_id).table(table_id)

# Delete the existing destination table if it exists
try:
    client.delete_table(table_ref)
    print(f"Deleted table {table_ref}")
except Exception as e:
    print(f"Table {table_ref} does not exist: {e}")

# Create a new table with the schema
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # API request


# Load DataFrame to BigQuery table
job = client.load_table_from_dataframe(df, table_ref)

# Wait for the load job to complete
job.result()

print(f"Loaded {job.output_rows} rows into {dataset_id}:{table_id}.")

Deleted table gentle-keyword-423715-j0.dummy.prediction-revenue
Loaded 1617 rows into dummy:prediction-revenue.
